`nullif` 函数的作用是将两个参数进行比较，如果两个参数相等，则返回 `null`，否则返回第一个参数。

In [19]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

In [3]:
sql = """
CREATE TABLE posts (
  id serial primary key, 
  title VARCHAR (255) NOT NULL, 
  excerpt VARCHAR (150), 
  body TEXT, 
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
  updated_at TIMESTAMP
);

INSERT INTO posts (title, excerpt, body)
VALUES
      ('test post 1','test post excerpt 1','test post body 1'),
      ('test post 2','','test post body 2'),
      ('test post 3', null ,'test post body 3')
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x293fc9c4880>

In [4]:
sql = """
SELECT 
  id, 
  title, 
  COALESCE (
    excerpt, 
    LEFT(body, 40)
  ) 
FROM 
  posts;
"""
run_sql(cursor, sql)

# excerpt 中既有 null，又有空字符串，可以使用 nullif 函数将其转换为 null

   id        title             coalesce
0   1  test post 1  test post excerpt 1
1   2  test post 2                     
2   3  test post 3     test post body 3


In [5]:
sql = """
SELECT 
  id, 
  title, 
  COALESCE (
    NULLIF (excerpt, ''), 
    LEFT (body, 40)
  ) 
FROM 
  posts;
"""
run_sql(cursor, sql)

   id        title             coalesce
0   1  test post 1  test post excerpt 1
1   2  test post 2     test post body 2
2   3  test post 3     test post body 3


`nullif` 防止除零错误

In [20]:
sql = """
CREATE TABLE members (
  id serial PRIMARY KEY, 
  first_name VARCHAR (50) NOT NULL, 
  last_name VARCHAR (50) NOT NULL, 
  gender SMALLINT NOT NULL -- 1: male, 2 female
);

INSERT INTO members (first_name, last_name, gender) 
VALUES 
  ('John', 'Doe', 1), 
  ('David', 'Dave', 1), 
  ('Bush', 'Lily', 2);
"""
cursor.execute(sql)

sql = """
SELECT * from members;
"""
run_sql(cursor, sql)

   id first_name last_name  gender
0   1       John       Doe       1
1   2      David      Dave       1
2   3       Bush      Lily       2


In [15]:
# 计算男女比例
sql = """
SELECT 
  (
    SUM (CASE WHEN gender = 1 THEN 1 ELSE 0 END) / SUM (CASE WHEN gender = 2 THEN 1 ELSE 0 END)
  ) * 100 AS "Male/Female ratio" 
FROM 
  members;
"""
run_sql(cursor, sql)

   Male/Female ratio
0                200


In [21]:
# 删除女性成员
# 计算男女比例
sql = """
DELETE FROM members 
WHERE gender = 2;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x293fc6000f0>

In [17]:
# 计算男女比例
sql = """
SELECT 
  (
    SUM (CASE WHEN gender = 1 THEN 1 ELSE 0 END) / SUM (CASE WHEN gender = 2 THEN 1 ELSE 0 END)
  ) * 100 AS "Male/Female ratio" 
FROM 
  members;
"""
run_sql(cursor, sql)

#　产生除零错误

DivisionByZero: 除以零

In [22]:
# 计算男女比例
sql = """
SELECT 
  (
    SUM (CASE WHEN gender = 1 THEN 1 ELSE 0 END) / NULLIF (
      SUM (CASE WHEN gender = 2 THEN 1 ELSE 0 END), 
      0
    )
  ) * 100 AS "Male/Female ratio" 
FROM 
  members;
"""
run_sql(cursor, sql)


  Male/Female ratio
0              None
